In [ ]:
from brayns import Client
from circuitexplorer import CircuitExplorer

In [ ]:
brayns = Client('localhost:5000')
ce = CircuitExplorer(brayns)

## Get region names

In [ ]:
import json

region_file = '/home/favreau/python/data/waxholm/brain_region.json'
f = open(region_file)
obj = json.load(f)

regions = dict()
def get_children(node, level):
    regions[node['id']] = [level, node['name']]
    try:
        for child in node['children']:
            get_children(child, level + 1)
    except KeyError as e: 
        pass

get_children(obj, 0)

## Load region meshes

In [ ]:
import glob
import os
nrrd_folder = '/home/favreau/python/data/waxholm/brain_regions/brain_region/'
mesh_files = glob.glob(nrrd_folder + '*.obj')
mesh_models = list()
for mesh_file in mesh_files:
    region_id = int(os.path.basename(mesh_file).split('.')[0])
    
    region = regions[region_id]
    region_level = region[0]
    region_name = region[1]
    if region_level == 3 and region_name.find('nerve') == -1:
#         print('Loading region %d: %s' % (region_id, region_name))
        model = brayns.add_model(path=mesh_file, name=region_name)
        mesh_models.append(model)

In [ ]:
for model in mesh_models:
    tf = model['transformation']
    tf['scale'] = [30, 30, 30]
    tf['translation'] = [-280,-650,-450]
    brayns.update_model(id=model['id'], transformation=tf)

In [ ]:
def set_model_color(model_id, color, intensity=1.0, opacity=1.0, shading_mode=ce.SHADING_MODE_DIFFUSE):
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    for i in range(nb_materials):
        
        if shading_mode==ce.SHADING_MODE_PERLIN:
            opacities.append(0.5)
            refraction_indices.append(1.5)
            specular_exponents.append(50)
            glossinesses.append(0.5)
            user_parameters.append(0.001)
        elif shading_mode==ce.SHADING_MODE_ELECTRON:
            opacities.append(0.75)
            refraction_indices.append(1.0)
            specular_exponents.append(5)
            glossinesses.append(1.0)
            user_parameters.append(1.0)
        else:
            opacities.append(0.5)
            refraction_indices.append(1.0)
            specular_exponents.append(50)
            glossinesses.append(1.0)
            user_parameters.append(1.0)

        c=color
        reflection_indices.append(0.0)
        diffuse_colors.append([c[0] * intensity,c[1] * intensity,c[2] * intensity])
        specular_colors.append([c[0] * intensity,c[1] * intensity,c[2] * intensity])
        simulation_data_casts.append(False)
        shading_modes.append(shading_mode)
        emissions.append(0)
        clips.append(True)

    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)

In [ ]:
import seaborn as sns
nb_models = len(mesh_models)
palette = sns.color_palette('rainbow', nb_models)
for i in range(nb_models):
    model = mesh_models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i], 
                    shading_mode=ce.SHADING_MODE_DIFFUSE,
                    intensity=2.0)

In [ ]:
from bioexplorer import BioExplorer, Widgets
be = BioExplorer()
w = Widgets(be)

In [36]:
w.display_model_visibility()

## S1 Circuit

In [ ]:
s1_config = '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/CircuitConfig_TC_WM'

In [ ]:
create_cache = False
cache_folder = '/home/favreau/python/data/brayns'
target_models = list()
for target in ['S1DZO', 'S1DZ', 'S1FL', 'S1HL', 'S1J', 'S1Sh', 'S1Tr', 'S1ULp']:
    cache_path = '%s/%s.brayns' % (cache_folder, target)
    if create_cache:
        model = ce.load_circuit(
            name=target,
            load_soma=True, radius_multiplier=5.0,
            use_sdf=False, load_axon=False, load_apical_dendrite=False, load_dendrite=False,
            circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
            density=100, path=s1_config, targets=[target])
        ce.save_model_to_cache(
            model_id=model['id'],
            path=cache_path
        )
        target_models.append(model)
    else:
        model = ce.load_circuit(name=target, path=cache_path)
        target_models.append(model)

In [ ]:
# model = ce.load_circuit(
#     name=target, load_soma=True, 
#     use_sdf=True, load_axon=False, load_apical_dendrite=True, load_dendrite=True,
#     circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NEURON_BY_LAYER,
#     density=0.1, path=s1_config, targets=['S1HL'])

# Could not open morphology file /gpfs/bbp.cscs.ch/project/proj83/morphologies/fixed_ais_L23PC_20201210/ascii/sm080930a1-5_idC_-_Scale_x1.000_y0.975_z1.000.h5

In [ ]:
import seaborn as sns
nb_models = len(target_models)
palette = sns.color_palette('rainbow', nb_models)
for i in range(nb_models):
    model = target_models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i],
                    shading_mode=ce.SHADING_MODE_DIFFUSE)

In [ ]:
for model in target_models:
    tf = model['transformation']
    tf['rotation'] = [0.0, 0.0, 1.0, 0.0]
    if model['name'] == 'S1HL':
        tf['translation'] = [-15000.0, 0.0, 0.0]
    else:
        tf['translation'] = [-7500.0, 0.0, 0.0]
    brayns.update_model(id=model['id'], transformation=tf)    

In [ ]:
box_model = ce.add_box(
    minCorner=[-32000, -20000, -10010], maxCorner=[20000, 20000, -10000],
    color=[0.5, 0.5, 0.5, 1.0]
)

In [ ]:
model_id = box_model['id']
ce.set_material_extra_attributes(model_id)
set_model_color(model_id, [0.2, 0.2, 0.2], 
                shading_mode=ce.SHADING_MODE_NONE)

## Camera

In [ ]:
brayns.set_camera(
    orientation=[0.008657863306720616, 0.004328919822465945, -1.4055102769718064e-05, 0.9999531497318196],
    position=[-7315.691873683525, -2666.139309893637, 25494.027720775728],
    target=[-7580.739032885286, -2136.0249149445644, -5116.074459866043]    
)